# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": nba (football, in [assets/nba.csv](assets/nba.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clear_data(string):
    return re.sub(r'\[[a-z]* [0-9]+\]', '', string)


def clear_nba_data(string):
    if re.search(r'\*|\+', string) is None:
        return string
    else:
        return re.sub(r'\*|\+', '', string)


def get_area(team,nfl_cities):
    for index, row in nfl_cities.iterrows():
        if team in index:
            return row['Metropolitan area']



def get_nfl_data():
    return outdf


def nhl_correlation():

    population_by_region = []
    win_loss_by_region = []

    nfldf= pd.read_csv("assets/nfl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]


    nfldf= nfldf[nfldf['year'] == 2018].drop([0, 5, 10, 15, 20, 25, 30, 35])

    population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]
    population = population.set_index('Metropolitan area')
    #print(population)
    cities['NFL'] = cities['NFL'].apply(lambda x: clear_data(x))
    nfl_cities = cities[['Metropolitan area', 'NFL']].set_index('NFL')
    nfl_cities = nfl_cities.drop(['—', ''], axis=0)

    nfldf['team'] = nfldf['team'].apply(lambda x: clear_nba_data(x))
 

    nfldf['area'] = nfldf['team'].apply(lambda x: x.split(" ")[-1])
    nfldf['area'] = nfldf['area'].apply(lambda x: get_area(x,nfl_cities))

    out = []
    #print(nfldf)
    for group, frame in nfldf.groupby('area'):
        total_wins = frame['W'].astype(int).sum()
        total_losses = frame['L'].astype(int).sum()
        total_matches = total_wins + total_losses
        ratio = total_wins / total_matches
        out_dict = {
            'Area': group,
            'Ratio': ratio
        }
        out.append(out_dict)

    newdf= pd.DataFrame(out)
    newdf= newdf.set_index('Area')

    outdf= pd.merge(newdf, population, how="inner", left_index=True, right_index=True)
    outdf['Population (2016 est.)[8]'] = pd.to_numeric(outdf['Population (2016 est.)[8]'])

    population_by_region = outdf['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = outdf['Ratio'].tolist()

    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return corr

nhl_correlation()

0.00492211214934941

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [99]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


def clear_data(string):
    return re.sub(r'\[[a-z]* [0-9]+\]', '', string)


def clear_nba_data(string):
    result= re.sub(r'\s*\*?\s*\(\d+\)$', '', string)
    return result


def get_area(team,nfl_cities):
    for index, row in nfl_cities.iterrows():
        if team in index:
            return row['Metropolitan area']


def get_nba_data():
    return outdf

def nba_correlation():
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhldfin the same order as cities["Metropolitan area"]
    nbadf= pd.read_csv("assets/nba.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    

    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]

    nbadf= nbadf[nbadf['year'] == 2018]  # get only 2018 stats no need of dropping rows
    population = cities[['Metropolitan area', 'Population (2016 est.)[8]']]

    #print(population)
    population = population.set_index('Metropolitan area')
    cities['NBA'] = cities['NBA'].apply(lambda x: clear_data(x))
    #print(cities['NBA'])
    nba_cities = cities[['Metropolitan area', 'NBA']].set_index('NBA')

    nba_cities = nba_cities.drop(['—', ''], axis=0)
    #print(nba_cities)
   
    
    nbadf['team'] = nbadf['team'].apply(lambda x: clear_nba_data(x))
    
    nbadf['area'] = nbadf['team'].apply(lambda x: x.split(" ")[-1])

    nbadf['area'] = nbadf['area'].apply(lambda x: get_area(x,nba_cities))

    #nbadf['area']=cities['Metropolitan area']
    print(nbadf)
    #print(population)
    out = []
    for group, frame in nbadf.groupby('area'):
        total_wins = np.sum(pd.to_numeric(frame['W']))
        total_losses = np.sum(pd.to_numeric(frame['L']))
        WL=np.sum(pd.to_numeric(frame['W/L%']))
        total_matches = total_wins + total_losses
        ratio = (total_wins / total_matches)
        out_dict = {
            'Area': group,
            'Ratio': ratio
        }
        out.append(out_dict)
    newdf= pd.DataFrame(out)
    newdf= newdf.set_index('Area')
    #print(newdf)
    #print(population)
    outdf= pd.merge(newdf, population, how="inner", left_index=True, right_index=True)
    outdf['Population (2016 est.)[8]'] = pd.to_numeric(outdf['Population (2016 est.)[8]'])
    population_by_region = outdf['Population (2016 est.)[8]'].tolist()
    win_loss_by_region = outdf['Ratio'].to_list()
    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return corr

nba_correlation()

                      team   W   L                 W/L%    GB   PS/G   PA/G  \
0          Toronto Raptors  59  23                 0.72     —  111.7  103.9   
1           Boston Celtics  55  27   0.6709999999999999   4.0  104.0  100.4   
2       Philadelphia 76ers  52  30                0.634   7.0  109.8  105.3   
3      Cleveland Cavaliers  50  32                 0.61   9.0  110.9  109.9   
4           Indiana Pacers  48  34                0.585  11.0  105.6  104.2   
5               Miami Heat  44  38                0.537  15.0  103.4  102.9   
6          Milwaukee Bucks  44  38                0.537  15.0  106.5  106.8   
7       Washington Wizards  43  39                0.524  16.0  106.6  106.0   
8          Detroit Pistons  39  43  0.47600000000000003  20.0  103.8  103.9   
9        Charlotte Hornets  36  46                0.439  23.0  108.2  108.0   
10         New York Knicks  29  53                0.354  30.0  104.5  108.0   
11           Brooklyn Nets  28  54                0.

-0.17657160252844614

In [8]:
import re

string1 = "Toronto Raptors* (1)"
string2 = "Orlando Magic (14)"

regex_pattern = r'\s*\*?\s*\(\d+\)$'
result1 = re.sub(regex_pattern, '', string1)
result2 = re.sub(regex_pattern, '', string2)

print(result1)  # Output: "Toronto Raptors"
print(result2)  # Output: "Orlando Magic"


Toronto Raptors
Orlando Magic


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [207]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1,\2', string1)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)

    utput_string = re.sub(r'[^a-zA-Z\s]+', '', output_string)
    
    # Convertir la chaîne en liste
    output_list = output_string.split(',')

    return output_list


def clear_nba_data(string):
    result= re.sub(r'\s*\*?\s*\(\d+\)$', '', string)
    return result


def get_area(team,mlb_cities):
    for index, row in mlb_cities.iterrows():
        if team in index:
            return row['Metropolitan area']
        
def get_ratio(out):
    for item in out:
        if item['Area'] == 'Atlanta':
            ratio_value = item['Ratio']
            break
    return ratio_value


def get_nba_data():
    return out_df

def mlb_correlation(): 
    population_by_region = []  # pass in metropolitan area population from cities
    win_loss_by_region = []  # pass in win/loss ratio from nhldfin the same order as cities["Metropolitan area"]
    mlb_df= pd.read_csv("assets/mlb.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    #print(cities)  

    cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
    cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)
    mlb_df= mlb_df[mlb_df['year'] == 2018]  # get only 2018 stats no need of dropping rows
    population = cities[['Metropolitan area', 'Population']]

    #print(population)
    #population = population.set_index('Metropolitan area')

    #cities=cities['MLB'].replace("",np.nan).replace("—",np.nan).dropna()

    #print(cities)
    #print(mlb_df)
    #####"cities['team'] = cities['MLB'].apply(lambda x: clear_data(x))
    #cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x))
 
    new_rows = []
    row=0
    WLratio=0

    cities['W-L%']=0
    citiestemp=cities.copy()
    for index, row in citiestemp.iterrows():
        cleaned_values = clear_data(row['MLB'])
        if len(cleaned_values) > 1:
            for valueTeam in cleaned_values[0:]:
                ville=row['Metropolitan area']
                pop=population.loc[population['Metropolitan area']==ville,'Population'].iloc[0]
                
                WLratio=mlb_df.loc[mlb_df['team'].str.contains(valueTeam),'W-L%'].iloc[0]
                print("WLratioooooooo"+str(WLratio))

                valeurs_correspondantes = cities.loc[cities['MLB'].str.contains(valueTeam), 'Metropolitan area'].iloc[0]    
                print('WLratio '+str(WLratio)+' '+str(valeurs_correspondantes)+' '+str(valueTeam))                                            
                new_rows.append({'Metropolitan area': ville, 'MLB': valueTeam, 'Population': pop, 'W-L%': str(WLratio)})
            cities=cities.drop(index)

            
        
        if len(cleaned_values) == 1:
            team=cleaned_values[0]
            #WLratio=mlb_df.loc[mlb_df['team'].str.contains('Mets'),'W-L%'].iloc[0]
            cities.loc[index,'MLB']=team
            #cities.loc[index,'W-L%'] = WLratio


    # Étendre le DataFrame avec les nouvelles lignes
    cities = cities.append(new_rows, ignore_index=True)

    print('cities')
    #print(cities)
    
    #cities['MLB'] = cities['MLB'].apply(lambda x: clear_data(x)[0])



    
    mlb_cities = cities[['Metropolitan area', 'MLB','Population','W-L%']].set_index('MLB')

    mlb_cities = mlb_cities.drop(['—', ''], axis=0)
    #mlb_cities['W-L%'] = mlb_cities['MLB'].apply(lambda x: mlb_df.loc[mlb_df['team'].str.contains(x),'W-L%'].iloc[0])
    #mlb_cities.reindex()
    for index, row in  mlb_cities.iterrows():
        WLratio=mlb_df.loc[mlb_df['team'].str.contains(index),'W-L%'].iloc[0]
        mlb_cities.loc[index,'W-L%'] = WLratio
    print(mlb_cities)
   
    #print(mlb_df) 
    #mlb_df['team'] = mlb_df['team'].apply(lambda x: clear_nba_data(x))
   
    #mlb_df['team'] = mlb_df['team'].apply(lambda x: x.split(" ")[-1])
    # print(mlb_df['team'])
      
    #mlb_df['city'] = mlb_df['team'].apply(lambda x: get_area(x,mlb_cities))
    #print(mlb_df['city'])
    #mlb_df['area']=cities['Metropolitan area']
    #print(mlb_df)   

    out = []
    ratio=0
    for group, frame in mlb_cities.groupby('Metropolitan area'):
        #total_wins = np.sum(pd.to_numeric(frame['W']))
        #total_losses = np.sum(pd.to_numeric(frame['L']))
        WL=np.mean(pd.to_numeric(frame['W-L%']))
        
        #total_matches = total_wins + total_losses
        ratio = WL#(total_wins / total_matches)
        rv=next((item['Ratio'] for item in out if item['City'] == group), 0)
        #print(rv)
        out_dict = {
            'City': group,
            'Ratio': next((item['Ratio'] for item in out if item['City'] == group), 0) + ratio
        }
        #print(next((item['Ratio'] for item in out if item['Area'] == group), 0))
        out.append(out_dict)
    #print(out)
    new_df= pd.DataFrame(out)
    new_df= new_df.set_index('City')
    #print(new_df)
    #print(population)
    
    out_df= pd.merge(new_df, population, how="inner", left_index=True, right_index=True)
    out_df['Population'] = pd.to_numeric(out_df['Population'])
    print(out_df['Population'].tolist())
    
    mlb_cities= mlb_cities.astype({'Population' : int, 'W-L%': float})
    population_by_region = mlb_cities['Population'] #out_df['Population'].tolist()
    win_loss_by_region = mlb_cities['W-L%'] #out_df['Ratio'].to_list()
    corr = stats.pearsonr(population_by_region, win_loss_by_region)[0]
    return corr


    
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlbdfin the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)


mlb_correlation()

WLratioooooooo0.617
WLratio 0.617 New York City Yankees
WLratioooooooo0.475
WLratio 0.475 New York City Mets
WLratioooooooo0.564
WLratio 0.564 Los Angeles Dodgers
WLratioooooooo0.494
WLratio 0.494 Los Angeles Angels
WLratioooooooo0.451
WLratio 0.451 San Francisco Bay Area Giants
WLratioooooooo0.599
WLratio 0.599 San Francisco Bay Area Athletics
WLratioooooooo0.583
WLratio 0.583 Chicago Cubs
WLratioooooooo0.383
WLratio 0.383 Chicago White Sox
cities
                   Metropolitan area Population   W-L%
MLB                                                   
Rangers            Dallas–Fort Worth    7233323  0.414
Nationals           Washington, D.C.    6131977  0.506
Phillies                Philadelphia    6070500  0.494
Red Sox                       Boston    4794447  0.667
Twins         Minneapolis–Saint Paul    3551036  0.481
Rockies                       Denver    2853077  0.558
Marlins        Miami–Fort Lauderdale    6066387  0.391
Diamondbacks                 Phoenix    4661537  0.5

C:\Users\Pierre\AppData\Local\Temp\ipykernel_1996\3521048124.py:95: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = cities.append(new_rows, ignore_index=True)


In [199]:
import re

input_string = "DodgersAngels"

# Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1\n\2', input_string)

# Supprimer les balises de note [note 10], [note 12], [note 14]
output_string = re.sub(r'\[note \d+\]', '', output_string)

print(output_string)


Dodgers
Angels


In [2]:
import re
import pandas as pd

# Fonction clear_data modifiée pour retourner deux valeurs
def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1\n\2', input_string)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)
    return output_string


# Exemple de données pour cities
data = {'Metropolitan area': ['City A', 'City B', 'City C'],
        'MLB': ['Team A', 'DodgersAngels[note 10]', 'Team C [note 12]']}
cities = pd.DataFrame(data)


print(cities)
# Appliquer la fonction clear_data avec gestion des deux valeurs retournées
cities['MLB'] = cities['MLB'].apply(lambda x: [clear_data(x), x.split(" ")[-1]][-1])

# Afficher le résultat
print(cities)



  Metropolitan area                     MLB
0            City A                  Team A
1            City B  DodgersAngels[note 10]
2            City C        Team C [note 12]


NameError: name 'input_string' is not defined

In [1]:
import pandas as pd
import re

# Fonction clear_data modifiée pour retourner une liste de valeurs distinctes
def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1,\2', string1)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)
    
    # Convertir la chaîne en liste
    output_list = output_string.split(',')

    return output_list

# Exemple de données pour cities
data = {'Metropolitan area': ['City A', 'City B', 'City C','Cityd'],
        'MLB': ['Tea','DodgersAngels[note 10]', 'Team [note 12]','KnicksNets']}
cities = pd.DataFrame(data)

# Appliquer la fonction clear_data et étendre le DataFrame avec les valeurs distinctes
new_rows = []
for index, row in cities.iterrows():
    cleaned_values = clear_data(row['MLB'])
    if len(cleaned_values) > 1:
        for value in cleaned_values[1:]:
            new_rows.append({'Metropolitan area': row['Metropolitan area'], 'MLB': value})
    row['MLB'] = cleaned_values[0]

# Étendre le DataFrame avec les nouvelles lignes
cities = cities.append(new_rows, ignore_index=True)

# Afficher le résultat
print(cities)
#print(cities.loc[cities['Metropolitan area']=='City A','MLB'][0])
valeurs_correspondantes = cities.loc[cities['MLB'].str.contains('Dorrrrrrrrd'), 'Metropolitan area'].iloc[0]
print(valeurs_correspondantes)

C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\1891125225.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = cities.append(new_rows, ignore_index=True)


  Metropolitan area      MLB
0            City A      Tea
1            City B  Dodgers
2            City C    Team 
3             Cityd   Knicks
4            City B   Angels
5             Cityd     Nets


IndexError: single positional indexer is out-of-bounds

In [1]:
import pandas as pd
import re

population_by_region = []  # pass in metropolitan area population from cities
win_loss_by_region = []  # pass in win/loss ratio from nhldfin the same order as cities["Metropolitan area"]
mlbdf= pd.read_csv("assets/mlb.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
    #print(cities)  

cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
mlbdf= mlbdf[mlbdf['year'] == 2018]  # get only 2018 stats no need of dropping rows
population = cities[['Metropolitan area', 'Population']]

print(mlbdf.loc[mlbdf['team'].str.contains('White'),'W-L%'].iloc[0])

0.383


In [146]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},inplace=True)

def clear_data(string1):
    # Remplacer "Nets" par "\nNets", "Angels" par "\nAngels", etc.
    output_string = re.sub(r'([a-zA-Z]+)([A-Z][a-z]+)', r'\1,\2', string1)

    # Supprimer les balises de note [note 10], [note 12], [note 14]
    output_string = re.sub(r'\[note \d+\]', '', output_string)

    utput_string = re.sub(r'[^a-zA-Z\s]+', '', output_string)
    
    # Convertir la chaîne en liste
    output_list = output_string.split(',')

    return output_list


def get_area(team,mlb_cities):
    for index, row in mlb_cities.iterrows():
        if team in index:
            return row['Metropolitan area']

def mlb_correlation():
    #print(cities['MLB'])
    team = cities['MLB'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    
    
    team['Metropolitan area'] = cities['Metropolitan area']
    

    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("", np.nan).replace("—", np.nan).dropna().reset_index().rename(columns={"value": "team"})
    team = pd.merge(team, cities, how='left', on='Metropolitan area').iloc[:, 1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team'] = team['team'].str.replace('\ Sox', 'Sox')
    team['team'] = team['team'].str.replace('[\w.]*\ ', '')

    df= pd.read_csv("assets/mlb.csv")
    df= df[df['year'] == 2018]
    df['team'] = df['team'].str.replace(r'[\*]', "")
    df['team'] = df['team'].str.replace(r'\(\d*\)', "")
    df['team'] = df['team'].str.replace(r'[\xa0]', "")
    df= df[['team', 'W-L%']]

    #df.rename(columns={"W-L%": "W/L%"}, inplace=True)
    df['team'] = df['team'].str.replace('\ Sox', 'Sox')
    df['team'] = df['team'].str.replace('[\w.]* ', '')
    df= df.astype({'team': str, 'W-L%': float})
    df=df.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    merge = pd.merge(team, df, 'outer', on='team')
    
    merge = merge.groupby('Metropolitan area').agg({'W-L%': np.nanmean, 'Population': np.nanmean})
    merge=merge.replace([np.inf, -np.inf], np.nan).dropna(axis=1)
    population_by_region = merge['Population']  # pass in metropolitan area population from cities
    win_loss_by_region = merge['W-L%']  # pass in win/loss ratio from dfin the same order as cities["Metropolitan area"]
    print(merge.iloc[1:20])

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()




                         W-L%  Population
Metropolitan area                        
Baltimore               0.290   2798886.0
Boston                  0.667   4794447.0
Chicago                 0.483   9512999.0
Cincinnati              0.414   2165139.0
Cleveland               0.562   2055612.0
Dallas–Fort Worth       0.414   7233323.0
Denver                  0.558   2853077.0
Detroit                 0.395   4297617.0
Houston                 0.636   6772470.0
Kansas City             0.358   2104509.0
Los Angeles             0.529  13310447.0
Miami–Fort Lauderdale   0.391   6066387.0
Milwaukee               0.589   1572482.0
Minneapolis–Saint Paul  0.481   3551036.0
New York City           0.546  20153634.0
Philadelphia            0.494   6070500.0
Phoenix                 0.506   4661537.0
Pittsburgh              0.509   2342299.0
San Diego               0.407   3317749.0


C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:41: FutureWarning: The default value of regex will change from True to False in a future version.
  team['team'] = team['team'].str.replace('\ Sox', 'Sox')
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  team['team'] = team['team'].str.replace('[\w.]*\ ', '')
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team'] = df['team'].str.replace(r'[\*]', "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['team'] = df['team'].str.replace(r'\(\d*\)', "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_3912\3231281658.py:48: FutureWarning: The default value of regex will change from True

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **nba** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nbadf=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nbadfin the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for nba"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [45]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1, [0, 3, 5, 6, 7, 8]]
cities.rename(columns={"Population (2016 est.)[8]": "Population"},
              inplace=True)
cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")

Big4='NFL'

def nfl_correlation(): 
    team = cities[Big4].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')
    team['Metropolitan area']=cities['Metropolitan area']
    team = pd.melt(team, id_vars=['Metropolitan area']).drop(columns=['variable']).replace("",np.nan).replace("—",np.nan).dropna().reset_index().rename(columns = {"value":"team"})
    team=pd.merge(team,cities,how='left',on = 'Metropolitan area').iloc[:,1:4]
    team = team.astype({'Metropolitan area': str, 'team': str, 'Population': int})
    team['team']=team['team'].str.replace('[\w.]*\ ','')
    
    df=pd.read_csv("assets/"+str.lower(Big4)+".csv")
    df= df[df['year'] == 2018]
    df['team'] = df['team'].str.replace(r'[\*]',"")
    df['team'] = df['team'].str.replace(r'\(\d*\)',"")
    df['team'] = df['team'].str.replace(r'[\xa0]',"")
    df= df[['team','W-L%']]
    df.rename(columns={"W-L%": "W/L%"},inplace=True)
    dropList=[]
    for i in range(df.shape[0]):
        row=df.iloc[i]
        if row['team']==row['W/L%']:
            dropList.append(i)
    df=df.drop(dropList)

    df['team'] = df['team'].str.replace('[\w.]* ','')
    df['team'] = df['team'].str.replace('+','')
    df= df.astype({'team': str,'W/L%': float})
    
    merge=pd.merge(team,df,'outer', on = 'team')
    merge=merge.groupby('Metropolitan area').agg({'W/L%': np.nanmean, 'Population': np.nanmean})

    population_by_region = merge['Population'] # pass in metropolitan area population from cities
    win_loss_by_region = merge['W/L%'] # pass in win/loss ratio from dfin the same order as cities["Metropolitan area"]   
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

C:\Users\Pierre\AppData\Local\Temp\ipykernel_4460\2689093289.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NFL'] = cities['NFL'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_4460\2689093289.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['MLB'] = cities['MLB'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_4460\2689093289.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NBA'] = cities['NBA'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_4460\2689093289.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  cities['NHL'] = cities['NHL'].str.replace(r"\[.*\]", "")
C:\Users\Pierre\AppData\Local\Temp\ipykernel_4460\2689093289.py:23: FutureWarning: The default value of regex will c

0.0042821414363930456

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlbdf=pd.read_csv("assets/mlb.csv")
nhldf=pd.read_csv("assets/nhl.csv")
nbadf=pd.read_csv("assets/nba.csv")
nbadf=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["nba","NBA"] should be the same as df.loc["NBA","nba"] and
    # df.loc["nba","nba"] should return np.nan
    sports = ['nba', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "nba"] - 0.80) <= 1e-2, "The MLB-nba p-value should be around 0.80"
    return p_values